This notebook analyzes data from https://github.com/HuthLab/deep-fMRI-dataset. To set up, see instructions in the `deep-fMRI-dataset` folder.

In [57]:
%load_ext autoreload
%autoreload 2
import datasets
import numpy as np
from os.path import join
from encoding.ridge_utils.SemanticModel import SemanticModel
from matplotlib import pyplot as plt
from typing import List
from sklearn.linear_model import RidgeCV, LogisticRegressionCV

from encoding.feature_spaces import em_data_dir, data_dir, results_dir

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load some data

In [61]:
dset = datasets.load_dataset('rotten_tomatoes')['train']
dset = dset.select(np.random.choice(len(dset), size=300, replace=False))
X = dset['text']
y = dset['label']

dset_test = datasets.load_dataset('rotten_tomatoes')['validation']
dset_test = dset_test.select(np.random.choice(len(dset_test), size=300, replace=False))
X_test = dset_test['text']
y_test = dset_test['label']

Using custom data configuration default
Reusing dataset rotten_tomatoes (/home/chansingh/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

Using custom data configuration default
Reusing dataset rotten_tomatoes (/home/chansingh/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

### Load the eng100 model

In [52]:
def get_vecs(X: List[str], save_location) -> np.ndarray:
    eng1000 = SemanticModel.load(join(em_data_dir, 'english1000sm.hf5'))
    # extract features
    X = [
        [word.encode('utf-8') for word in sentence.split(' ')]
        for sentence in X
    ]
    feats = eng1000.project_stims(X)
    return feats

def get_embs(feats: np.ndarray, save_location) -> np.ndarray:
    weights_npz = np.load(join(save_location, 'weights.npz'))
    corrs_val = np.load(join(save_location, 'corrs.npz'))['arr_0']
    

    weights = weights_npz['arr_0']
    N_DELAYS = 4
    # pretty sure this is right, but might be switched...
    weights = weights.reshape(N_DELAYS, -1, feats.shape[-1]) 
    # delays for coefs are not stored next to each other!! (see cell 25 file:///Users/chandan/Downloads/speechmodeltutorial-master/SpeechModelTutorial%20-%20Pre-run.html)
    # weights = weights.reshape(-1, N_DELAYS, feats.shape[-1]) 
    weights = weights.mean(axis=0).squeeze() # mean over delays dimension...
    embs = feats @ weights.T

    # subselect repr
    perc = np.percentile(corrs_val, 95)
    idxs = (corrs_val > perc)
    # print('emb dim', idxs.sum(), 'val corr cutoff', perc)
    embs = embs[:, idxs]

    return embs

In [62]:
save_location = join(results_dir, 'eng1000', 'UTS03')

print('extracting train embs...')
vecs = get_vecs(X, save_location)
embs = get_embs(vecs, save_location)

print('extracting test embs...')
X_test = dset_test['text']
vecs_test = get_vecs(X_test, save_location)
embs_test = get_embs(vecs, save_location)

extracting train embs...
extracting test embs...


In [63]:
m = LogisticRegressionCV()

In [64]:
m.fit(vecs, y)

/home/chansingh/.embgam/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/chansingh/.embgam/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

LogisticRegressionCV()

In [65]:
m.score(vecs_test, y_test)

0.6433333333333333

In [66]:
m = LogisticRegressionCV()

In [68]:
m.fit(embs, y)

/home/chansingh/.embgam/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/chansingh/.embgam/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

LogisticRegressionCV()

In [69]:
m.score(embs_test, y_test)

0.4533333333333333